In [ ]:
# !pip install sqlalchemy

In [134]:
import sqlalchemy as db
import pandas as pd
import numpy as np
import numpy
import re
import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer
import spacy
NLP = spacy.load("en_core_web_lg")

In [135]:
avatar_id = '1'

In [136]:
sql_url = "mysql+mysqlconnector://root@localhost/toia"

engine = db.create_engine(sql_url)
connection = engine.connect()
metadata = db.MetaData()
videos = db.Table('video', metadata, autoload=True, autoload_with=engine)

avatar_kb = db.select([videos]).where(
    videos.columns.toia_id == avatar_id,
    videos.columns.private == 0,
    videos.columns.type.notin_(["filler", "exit"])
)

ResultProxy = connection.execute(avatar_kb)
ResultSet = ResultProxy.fetchall()


In [137]:
df_avatar = pd.DataFrame(ResultSet, 
             columns=[
                 'id_video', 
                 'type', 
                 'toia_id', 
                 'idx', 
                 'private', 
                 'question', 
                 'answer', 
                 'language', 
                 'likes', 
                 'views',
             ])

In [138]:
df_avatar

,id_video,type,toia_id,idx,private,question,answer,language,likes,views
0,0b2,answer,1,2,0,What is your favorite sport?,I love soccer!,en-US,2,5
1,a7c,answer,1,3,0,What do you study?,I study Computer Science.,en-US,10,34
2,ef1,answer,1,1,0,How are you?,I am fine thanks!,en-US,5,14
3,r2a,answer,1,7,0,What do you do?,I am doing a Ph.D.,en-US,4,20
4,r2b,answer,1,8,0,How old are you?,I'm 35. Age last birthday,en-US,4,20
5,r2c,answer,1,9,0,Tell me something interesting?,I wrote a book about the ethics of artificial ...,en-US,4,20
6,r2e,greeting,1,16,0,hey,how's it going?,en-US,4,20
7,r2f,greeting,1,11,0,Hello,Hi!,en-US,4,20
8,r2h,no-answer,1,12,0,,"Sorry, I didn't get that",en-US,4,20
9,r2m,greeting,1,6,0,Howdy,"hey, I'm all right",en-US,4,20


In [139]:
docs = NLP.pipe(df_avatar['question'].values)
for doc in docs:
    print("--- next doc ---")
    print(doc)
    for token in doc:
        print(token.text, token.pos_, token.tag_)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

--- next doc ---
What is your favorite sport?
What PRON WP
is AUX VBZ
your PRON PRP$
favorite ADJ JJ
sport NOUN NN
? PUNCT .
--- next doc ---
What do you study?
What PRON WP
do AUX VBP
you PRON PRP
study VERB VB
? PUNCT .
--- next doc ---
How are you?
How ADV WRB
are AUX VBP
you PRON PRP
? PUNCT .
--- next doc ---
What do you do?
What PRON WP
do AUX VBP
you PRON PRP
do VERB VB
? PUNCT .
--- next doc ---
How old are you?
How ADV WRB
old ADJ JJ
are AUX VBP
you PRON PRP
? PUNCT .
--- next doc ---
Tell me something interesting?
Tell VERB VB
me PRON PRP
something PRON NN
interesting ADJ JJ
? PUNCT .
--- next doc ---
hey
hey INTJ UH
--- next doc ---
Hello
Hello INTJ UH
--- next doc ---

--- next doc ---
Howdy
Howdy INTJ UH
--- next doc ---
Do you like sushi?
Do AUX VBP
you PRON PRP
like VERB VB
sushi NOUN NN
? PUNCT .
--- next doc ---

--- next doc ---
Do you like swimming?
Do AUX VBP
you PRON PRP
like VERB VB
swimming VERB VBG
? PUNCT .


In [140]:
doc = NLP("hey, hi!")

In [141]:
['INTJ', 'UH'] in [[token.pos_, token.tag_] for token in doc]

True

In [142]:
set([token.tag_ for token in doc])

{',', '.', 'UH'}

In [143]:
df_avatar[df_avatar['type'] == "greeting"].sample(n=1)['answer'].values[0]

"hey, I'm all right"

In [144]:
ps = SnowballStemmer('english')

def preprocess(text):
            # Stem and remove stopwords
            text = re.sub('[^a-zA-Z]', ' ', text)
            text = text.lower()
            text = text.split()
            text = [ps.stem(word) for word in text]  # if not word in set(stopwords.words('english'))]
            return ' '.join(text)


def toia_answer(query, dataset, k=1):
    doc = NLP(query)
    # if Greeting, greet
    if ['INTJ', 'UH'] in [[token.pos_, token.tag_] for token in doc]:
        if dataset[dataset['type'] == "greeting"].shape[0] > 0:
            answers = dataset[dataset['type'] == "greeting"].sample(n=1)
            return answers['answer'].values[0], answers['id_video'].values[0]
        else:
            answers = dataset[dataset['type'] == "no-answer"].sample(n=1)
            return answers['answer'].values[0], answers['id_video'].values[0], "No greetings recorded"

    querycorpus = []
    for i in range(0, len(df_avatar)):
        userquestion = preprocess(df_avatar['question'][i])
        querycorpus.append(userquestion)

    # Creating the Bag of Words model with TFIDF and calc cosine_similarity
    vectorizer = CountVectorizer(decode_error="replace")
    # this is needed to get the attribute vocabulary_
    vec_train = vectorizer.fit_transform(querycorpus)
    training_vocabulary = vectorizer.vocabulary_
    transformer = TfidfTransformer()
    trainingvoc_vectorizer = CountVectorizer(
        decode_error="replace", vocabulary=training_vocabulary)
    tfidf_querycorpus = TfidfVectorizer().fit_transform(querycorpus)

    tfidf_userquestion = transformer.fit_transform(
        trainingvoc_vectorizer.fit_transform(
            numpy.array([
                preprocess(query)
            ])))
    cosine_similarities = cosine_similarity(tfidf_userquestion, tfidf_querycorpus)
    related_docs_indices = (-cosine_similarities[0]).argsort()
    sorted_freq = cosine_similarities[0][related_docs_indices]

    # note for this distance the problem we had befor with inf, we have now with 0. Again we decide
    # to make the prediction a bit random. This could be adjusted to remove any 0 distance and
    # pick the only ones left if any, and if none predict 1.

    if sum(sorted_freq) == 0:
        answers = dataset[dataset['type'] == "no-answer"].sample(n=1)
        return answers['answer'].values[0], answers['id_video'].values[0], "tfidf sim all 0"

    elif sorted_freq[0] > 0.7:  #(the top sorted freq is the max)
        if sorted_freq[k-1] != sorted_freq[k] or sorted_freq[k-1] == sorted_freq[k] == 0:
            selected = related_docs_indices[:k]
            return dataset.iloc[selected[0]]['answer'], dataset.iloc[selected[0]]['id_video'], f"tfidf sim: {sorted_freq[:k]}"
        else:
            indeces = numpy.where(numpy.roll(sorted_freq, 1) != sorted_freq)
            selected = related_docs_indices[:indeces[0][indeces[0] >= k][0]]
            return dataset.iloc[selected[0]]['answer'], dataset.iloc[selected[0]]['id_video'], f"tfidf sim: {sorted_freq[:k]}"

    else:
        docs = NLP.pipe(dataset['question'].values)
        cosine_similarities = [calculate_similarity(query, doc.text) for doc in docs]
        if max(cosine_similarities) > 0.5:
            related_docs_indices = np.argsort(cosine_similarities)[::-1]
            selected = related_docs_indices[:k][0]
            return dataset.iloc[selected]['answer'], dataset.iloc[selected]['id_video'], f"spaCy sim: {cosine_similarities[selected]}"
        else:
            answers = dataset[dataset['type'] == "no-answer"].sample(n=1)
            return answers['answer'].values[0], answers['id_video'].values[0], f"spaCy sim: {max(cosine_similarities)}"

In [145]:
toia_answer("hola cachina", df_avatar)

("I don't have an answer for that", 'r2y')

***

## Experiment with spaCy

In [146]:
def process_text(text):
    doc = NLP(text.lower())
    result = []
    for token in doc:
        if token.text in NLP.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return " ".join(result)

In [147]:
def calculate_similarity(text1, text2):
    base = NLP(process_text(text1))
    compare = NLP(process_text(text2))
    return base.similarity(compare)

In [148]:
docs = NLP.pipe(df_avatar['question'].values)
similar_list = [calculate_similarity("Yo!", doc.text) for doc in docs]
similar_list

<ipython-input-147-5b3d2fc83f3d>:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return base.similarity(compare)


[0.1960290722844779,
 0.07379277395621531,
 0.0,
 0.0,
 0.3354120586746895,
 0.29740283310352034,
 0.5982720020715394,
 0.4097530789445466,
 0.0,
 0.33224449180378135,
 0.3386126303165402,
 0.0,
 0.30997427744569134]

In [149]:
[(df_avatar.loc[j, 'question'], i) for
 i, j in zip(
     np.sort(similar_list)[::-1], 
     np.argsort(similar_list)[::-1])]

[('hey', 0.5982720020715394),
 ('Hello', 0.4097530789445466),
 ('Do you like sushi?', 0.3386126303165402),
 ('How old are you?', 0.3354120586746895),
 ('Howdy', 0.33224449180378135),
 ('Do you like swimming?', 0.30997427744569134),
 ('Tell me something interesting?', 0.29740283310352034),
 ('What is your favorite sport?', 0.1960290722844779),
 ('What do you study?', 0.07379277395621531),
 ('', 0.0),
 ('', 0.0),
 ('What do you do?', 0.0),
 ('How are you?', 0.0)]

### Test new function

In [150]:
dataset[dataset['type'] == "abal"].shape[0] == 0

True

In [189]:
# query = "what's your name"
# dataset = df_avatar.copy()
# k = 1

def toia_answer_new(query, dataset, k=1):
    doc = NLP(query)
    # if Greeting, greet
    if ['INTJ', 'UH'] in [[token.pos_, token.tag_] for token in doc]:
        if dataset[dataset['type'] == "greeting"].shape[0] > 0:
            answers = dataset[dataset['type'] == "greeting"].sample(n=1)
            return answers['answer'].values[0], answers['id_video'].values[0]
        else:
            answers = dataset[dataset['type'] == "no-answer"].sample(n=1)
            return answers['answer'].values[0], answers['id_video'].values[0], "No greetings recorded"

    querycorpus = []
    for i in range(0, len(df_avatar)):
        userquestion = preprocess(df_avatar['question'][i])
        querycorpus.append(userquestion)

    # Creating the Bag of Words model with TFIDF and calc cosine_similarity
    vectorizer = CountVectorizer(decode_error="replace")
    # this is needed to get the attribute vocabulary_
    vec_train = vectorizer.fit_transform(querycorpus)
    training_vocabulary = vectorizer.vocabulary_
    transformer = TfidfTransformer()
    trainingvoc_vectorizer = CountVectorizer(
        decode_error="replace", vocabulary=training_vocabulary)
    tfidf_querycorpus = TfidfVectorizer().fit_transform(querycorpus)

    tfidf_userquestion = transformer.fit_transform(
        trainingvoc_vectorizer.fit_transform(
            numpy.array([
                preprocess(query)
            ])))
    cosine_similarities = cosine_similarity(tfidf_userquestion, tfidf_querycorpus)
    related_docs_indices = (-cosine_similarities[0]).argsort()
    sorted_freq = cosine_similarities[0][related_docs_indices]

    # note for this distance the problem we had befor with inf, we have now with 0. Again we decide
    # to make the prediction a bit random. This could be adjusted to remove any 0 distance and
    # pick the only ones left if any, and if none predict 1.

    if sum(sorted_freq) == 0:
        answers = dataset[dataset['type'] == "no-answer"].sample(n=1)
        return answers['answer'].values[0], answers['id_video'].values[0], "tfidf sim all 0"

    elif sorted_freq[0] > 0.7:  #(the top sorted freq is the max)
        if sorted_freq[k-1] != sorted_freq[k] or sorted_freq[k-1] == sorted_freq[k] == 0:
            selected = related_docs_indices[:k]
            return dataset.iloc[selected[0]]['answer'], dataset.iloc[selected[0]]['id_video'], f"tfidf sim: {sorted_freq[:k]}"
        else:
            indeces = numpy.where(numpy.roll(sorted_freq, 1) != sorted_freq)
            selected = related_docs_indices[:indeces[0][indeces[0] >= k][0]]
            return dataset.iloc[selected[0]]['answer'], dataset.iloc[selected[0]]['id_video'], f"tfidf sim: {sorted_freq[:k]}"

    else:
        docs = NLP.pipe(dataset['question'].values)
        cosine_similarities = [calculate_similarity(query, doc.text) for doc in docs]
        if max(cosine_similarities) > 0.5:
            related_docs_indices = np.argsort(cosine_similarities)[::-1]
            selected = related_docs_indices[:k][0]
            return dataset.iloc[selected]['answer'], dataset.iloc[selected]['id_video'], f"spaCy sim: {cosine_similarities[selected]}"
        else:
            answers = dataset[dataset['type'] == "no-answer"].sample(n=1)
            return answers['answer'].values[0], answers['id_video'].values[0], f"spaCy sim: {max(cosine_similarities)}"

            
# For testing function, convert all 'return' to 'return_a ='
# return_a  

In [192]:
def run_toia(dataset):
    print("TOIA starts")

    while True:
        query = input('> ')
        if query == "stop":
            break

        output = toia_answer_new(query, dataset)
        if output is None:
            break

        print(output)

In [193]:
run_toia(df_avatar)

TOIA starts


>  what's your name


<ipython-input-147-5b3d2fc83f3d>:4: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return base.similarity(compare)


("I don't have an answer for that", 'r2y', 'spaCy sim: 1.0')


>  how are you


...

>  never mind


("I don't have an answer for that", 'r2y', 'tfidf sim all 0')


>  what is your favorite food


('I love soccer!', '0b2', 'tfidf sim: [0.87910785]')


>  stop


***

## WIP: Can see if an ELIZA-like algorithm can help

https://github.com/wadetb/eliza


***